In [1]:
import pandas as pd
import numpy as np


## Load Origin and Destination data separately 

In [9]:
#load data
files = [
    "archive/Combined_Flights_2018.parquet", 
    "archive/Combined_Flights_2019.parquet", 
    "archive/Combined_Flights_2022.parquet"
]

origin_columns = [
    "OriginCityName",
    "OriginAirportID",
    "OriginCityMarketID",
    "Origin"
]
dest_columns = [
    "DestCityName",
    "DestAirportID",
    "DestCityMarketID",
    "Dest"
]


origin_df = pd.concat(
    [
        pd.read_parquet(files[0], columns = origin_columns, engine="fastparquet"),
        pd.read_parquet(files[1], columns = origin_columns, engine="fastparquet"),
        pd.read_parquet(files[2], columns = origin_columns, engine="fastparquet")
    ],
    axis = 0)
dest_df = pd.concat(
    [
        pd.read_parquet(files[0], columns = dest_columns, engine="fastparquet"),
        pd.read_parquet(files[1], columns = dest_columns, engine="fastparquet"),
        pd.read_parquet(files[2], columns = dest_columns, engine="fastparquet")
    ],
    axis = 0)

## Merge Origin and Destination data into a single dataset of just Airports

In [10]:
#drop missing values
origin_df.dropna(inplace=True)
dest_df.dropna(inplace=True)

#change columns to look like same dataframe
origin_df['CityName'] = origin_df['OriginCityName']
origin_df['AirportID'] = origin_df['OriginAirportID']
origin_df['CityMarketID'] = origin_df['OriginCityMarketID']
origin_df['AirportName'] = origin_df['Origin']

dest_df['CityName'] = dest_df['DestCityName']
dest_df['AirportID'] = dest_df['DestAirportID']
dest_df['CityMarketID'] = dest_df['DestCityMarketID']
dest_df['AirportName'] = dest_df['Dest']

origin_df.drop(origin_columns, axis=1, inplace=True)
dest_df.drop(dest_columns, axis=1, inplace=True)

#merge origin and dest dataframes
df=pd.concat([origin_df, dest_df], ignore_index=True)
origin_df = pd.DataFrame()
dest_df = pd.DataFrame()

## Reduce the dataset down to distinct values

In [19]:
#get all unique airport IDs
airportIDs = pd.unique(df['AirportID'])

#get rest of the data and form new dataframe
data_extract = pd.DataFrame()

cityname_list = list()
airportid_list = list()
citymarketid_list = list()
airportname_list = list()

for airport in airportIDs:
    row = df[df['AirportID'] == airport].iloc[0]
    cityname_list.append(row[0])
    airportid_list.append(row[1])
    citymarketid_list.append(row[2])
    airportname_list.append(row[3])

final_df=pd.DataFrame({
    'CityName': cityname_list,
    'AirportID': airportid_list,
    'CityMarketID': citymarketid_list,
    'AirportName': airportname_list
})

In [22]:
#save registry of airports to file
# final_df.to_excel("data/excel_airport_registry.xlsx")
final_df.to_parquet("data/airport_registry.parquet")